In [1]:
#Импортируем необходиимых библиотек для реализации парсинга
from selenium import webdriver
from selenium_stealth import stealth # делает похожим на настоящего пользователя
from bs4 import BeautifulSoup
from selenium.webdriver.common.keys import Keys
from time import sleep
from tqdm import tqdm
import pandas as pd
import numpy as np

In [84]:
options = webdriver.ChromeOptions()
options.add_argument("start_maximized")

options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option('useAutomationExtension', False)
driver = webdriver.Chrome(options=options, executable_path=r"C:\Python\chromedriver.exe")
#chromedriver должен соответствовать актуальной версии chrom, тут используется версия chrom 107.0.5304.121
#версия chromedriver - 107.0.5304.62 (все работает!)

stealth(driver,
        languages=["en-US", "en"],
        vendor="Google Inc.",
        platform="Win32",
        webgl_vendor="Intel Iris OpenGL Engine",
        fix_hairline=True,
        )

link_all = []

#Сайт выдает 532 стр судов РФ, но показывает только 200 стр
#Выгпузка ссылок делается только по годам, так как там, где информация не заполнена, данные отсутствуют
#Выгружаем ссылки по судам до 1989 года постройки (194 стр, 3880 судна)
for page in tqdm(range(1, 195, 1), 'Обработанно страниц'):
    url = f"https://www.vesselfinder.com/ru/vessels?page={page}&minYear=1800&maxYear=1989&flag=RU" #загружаем русскоязычную версию сайта
    driver.get(url)
    
    #Собираем все ссылки
    for j in range(1, 21, 1):
        try:
            link = driver.find_element_by_class_name('results').find_elements_by_tag_name('tr')[j].find_elements_by_tag_name('td') #выгружаем класс
            links = link[0].find_element_by_tag_name('a').get_attribute("href") #вытаскиваем ссылку
            link_all.append(links) #складываем ее в словарь
        except:
            links = 0
            link_all.append(links)
            continue

#Выгружаем ссылки по судам с 1990 по 2022 года постройки (105 стр, 2093 судна)
for page in tqdm(range(1, 106, 1), 'Обработанно страниц'):
    url = f"https://www.vesselfinder.com/ru/vessels?page={page}&minYear=1990&maxYear=2022&flag=RU" #загружаем русскоязычную версию сайта
    driver.get(url)
    
    #Собираем все ссылки
    if page == 105:
        for j in range(1, 14, 1):
            try:
                link = driver.find_element_by_class_name('results').find_elements_by_tag_name('tr')[j].find_elements_by_tag_name('td') #выгружаем класс
                links = link[0].find_element_by_tag_name('a').get_attribute("href") #вытаскиваем ссылку
                link_all.append(links) #складываем ее в словарь
            except:
                links = 0
                link_all.append(links)
                continue
    else:
        for j in range(1, 21, 1):
            try:
                link = driver.find_element_by_class_name('results').find_elements_by_tag_name('tr')[j].find_elements_by_tag_name('td') #выгружаем класс
                links = link[0].find_element_by_tag_name('a').get_attribute("href") #вытаскиваем ссылку
                link_all.append(links) #складываем ее в словарь
            except:
                links = 0
                link_all.append(links)
                continue

C:\Users\skapr\AppData\Local\Temp\ipykernel_13692\1372113456.py:6: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(options=options, executable_path=r"C:\Python\chromedriver.exe")
Обработанно страниц:   0%|                                                                     | 0/194 [00:00<?, ?it/s]C:\Users\skapr\AppData\Local\Temp\ipykernel_13692\1372113456.py:30: DeprecationWarning: find_element_by_class_name is deprecated. Please use find_element(by=By.CLASS_NAME, value=name) instead
  link = driver.find_element_by_class_name('results').find_elements_by_tag_name('tr')[j].find_elements_by_tag_name('td') #выгружаем класс
C:\Users\skapr\AppData\Local\Temp\ipykernel_13692\1372113456.py:30: DeprecationWarning: find_elements_by_tag_name is deprecated. Please use find_elements(by=By.TAG_NAME, value=name) instead
  link = driver.find_element_by_class_name('results').find_elements_by_tag_name('tr')[j].find_elements_by_tag_nam

In [86]:
len(link_all)

5973

In [87]:
#создаем DF
df_link = pd.DataFrame(link_all)
df_link

,0
0,https://www.vesselfinder.com/ru/vessels/detail...
1,https://www.vesselfinder.com/ru/vessels/detail...
2,https://www.vesselfinder.com/ru/vessels/detail...
3,https://www.vesselfinder.com/ru/vessels/detail...
4,https://www.vesselfinder.com/ru/vessels/detail...
...,...
5968,https://www.vesselfinder.com/ru/vessels/detail...
5969,https://www.vesselfinder.com/ru/vessels/detail...
5970,https://www.vesselfinder.com/ru/vessels/detail...
5971,https://www.vesselfinder.com/ru/vessels/detail...


In [88]:
#Сохраняем в Excel
writer = pd.ExcelWriter('Ссылки_vesselfinder.xlsx')
df_link.to_excel(writer, 'link')
writer.save()

In [90]:
# Загрузка имо
lnk=pd.read_excel('D:\Стас\Ученье - Свет\Оценка\Аспирантура\Видео по NN, Data mining, ABC и т.д\Распределение Гаусса, регрессия\Python\!Наработки\Парсинг по судам/Ссылки_vesselfinder.xlsx', sheet_name = 0)
lnk.head(20)

,Ссылки
0,https://www.vesselfinder.com/ru/vessels/detail...
1,https://www.vesselfinder.com/ru/vessels/detail...
2,https://www.vesselfinder.com/ru/vessels/detail...
3,https://www.vesselfinder.com/ru/vessels/detail...
4,https://www.vesselfinder.com/ru/vessels/detail...
5,https://www.vesselfinder.com/ru/vessels/detail...
6,https://www.vesselfinder.com/ru/vessels/detail...
7,https://www.vesselfinder.com/ru/vessels/detail...
8,https://www.vesselfinder.com/ru/vessels/detail...
9,https://www.vesselfinder.com/ru/vessels/detail...


Последние цифры в ссылке (7913622) https://www.vesselfinder.com/ru/vessels/details/7913622 и есть ИМО, поэтому бегать по ссылкам и собирать ИМО нет необходимости.